In [ ]:
import zipfile
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import MobileNetV2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

LOAD DATA

In [ ]:
local_file = 'train.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/train')
zip_ref.close()

In [ ]:
local_file = 'test.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/test')
zip_ref.close()

In [ ]:
train_dir = '/content/drive/MyDrive/CAPSTONE/train'
validation_dir = '/content/drive/MyDrive/CAPSTONE/test'

LOAD TRANFER LEARNING

In [ ]:
tfmodel =  Xception(weights = 'imagenet',
                                input_shape = (150, 150, 3),
                                include_top = False
                                )


In [ ]:
for layer in tfmodel.layers:
    layer.trainable = False
last_layer =  tfmodel.output

DATA AUGMENTATION

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
train_generator=  train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'
)

In [ ]:
val_datagen = ImageDataGenerator(rescale = 1.0/255.0)

In [ ]:
val_generator = val_datagen.flow_from_directory(validation_dir,
                                                target_size = (150, 150),
                                                batch_size = 32,
                                                class_mode = 'categorical'
                                                )

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

MODEL Xception

In [ ]:
x = last_layer
x = layers.Flatten()(x)
x = layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-3, seed=42),
                       beta_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-3, seed=42))(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-3, seed=42),
                       beta_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-3, seed=42))(x)
x = layers.Dense(7, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)

In [ ]:
model = Model(tfmodel.input, x)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.9
)

In [ ]:
model.compile(
        optimizer=RMSprop(learning_rate=lr_schedule),
        loss='categorical_crossentropy',
        metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True
)

In [ ]:
model.summary()

In [ ]:
history = model.fit(
        train_generator,
        epochs=50,
        validation_data = val_generator,
        shuffle=False,
        # callbacks=[early_stopping],
        verbose = 1)

In [ ]:
validation_loss, validation_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {validation_accuracy}")

In [ ]:
# Plot training and validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training and validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
num_images_to_visualize = 6

for i in range(num_images_to_visualize):
    # Get a batch of validation data
    batch = val_generator.next()

    # Extract images and labels from the batch
    images, labels = batch

    # Make predictions
    predictions = model.predict(images)

    # Get the index of the predicted class for each image
    predicted_classes = np.argmax(predictions, axis=1)

    # Display the images along with their actual and predicted labels
    for j in range(images.shape[0]):
        plt.imshow(images[j])
        actual_label = np.argmax(labels[j])
        predicted_label = predicted_classes[j]

        # Check if the prediction is correct or not
        prediction_correct = actual_label == predicted_label

        plt.title(f"Actual: {actual_label}, Predicted: {predicted_label}, Correct: {prediction_correct}")
        plt.show()